In [1]:
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,random_split
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import typing
import onnxruntime as ort
import speech_recognition as sr
from torchvision import transforms
from PIL import Image
import threading
import time


In [2]:
# Veri yolları
data= r"İşaretDiliVideo" 

In [11]:
#frame kodu
import cv2
import os
import json
import unicodedata

# Türkçe karakterleri ASCII uyumlu hale getir
def normalize_filename(text):
    replacements = {
        "ç": "c", "ğ": "g", "ı": "i", "ö": "o", "ş": "s", "ü": "u",
        "Ç": "C", "Ğ": "G", "İ": "I", "Ö": "O", "Ş": "S", "Ü": "U"
    }
    for turkish_char, ascii_char in replacements.items():
        text = text.replace(turkish_char, ascii_char)
    return text

def process_video(video_path, label, output_folder, json_data):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Video açılırken hata oluştu: {video_path}")
        return
    
    frame_count = 0
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_filename = f"{label}_{frame_count}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)

        try:
            cv2.imwrite(frame_path, frame)
            print(f"Görsel kaydedildi: {frame_path}")
        except Exception as e:
            print(f"Resim kaydedilirken hata oluştu: {e}")
            continue

        frames.append({
            "image_path": frame_path,
            "label": label
        })

        frame_count += 1

    cap.release()
    json_data[label] = frames
    print(f"{label} etiketli video işlendi, {frame_count} kare kaydedildi.")

def save_json(json_data, output_json_path):
    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)
        print(f"Veriler {output_json_path} dosyasına kaydedildi.")
    except Exception as e:
        print(f"JSON dosyasına yazılırken hata oluştu: {e}")

def process_videos(video_folder, output_folder, json_output_path):
    json_data = {}

    for video_name in os.listdir(video_folder):
        if video_name.endswith(".mp4"):
            label = os.path.splitext(video_name)[0]
            label_normalized = normalize_filename(label)  # Türkçe karakterleri sadeleştir

            video_path = os.path.join(video_folder, video_name)
            print(f"{video_name} işleniyor... (Etiket: {label_normalized})")
            process_video(video_path, label_normalized, output_folder, json_data)

    save_json(json_data, json_output_path)

# Ana çağrı
if __name__ == "__main__":
    video_folder = r"İşaretDiliVideo"
    output_folder = r"OutputImages"
    json_output_path = r"output_data.json"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    process_videos(video_folder, output_folder, json_output_path)


Anadolu.mp4 işleniyor... (Etiket: Anadolu)
Görsel kaydedildi: OutputImages\Anadolu_0.jpg
Görsel kaydedildi: OutputImages\Anadolu_1.jpg
Görsel kaydedildi: OutputImages\Anadolu_2.jpg
Görsel kaydedildi: OutputImages\Anadolu_3.jpg
Görsel kaydedildi: OutputImages\Anadolu_4.jpg
Görsel kaydedildi: OutputImages\Anadolu_5.jpg
Görsel kaydedildi: OutputImages\Anadolu_6.jpg
Görsel kaydedildi: OutputImages\Anadolu_7.jpg
Görsel kaydedildi: OutputImages\Anadolu_8.jpg
Görsel kaydedildi: OutputImages\Anadolu_9.jpg
Görsel kaydedildi: OutputImages\Anadolu_10.jpg
Görsel kaydedildi: OutputImages\Anadolu_11.jpg
Görsel kaydedildi: OutputImages\Anadolu_12.jpg
Görsel kaydedildi: OutputImages\Anadolu_13.jpg
Görsel kaydedildi: OutputImages\Anadolu_14.jpg
Görsel kaydedildi: OutputImages\Anadolu_15.jpg
Görsel kaydedildi: OutputImages\Anadolu_16.jpg
Görsel kaydedildi: OutputImages\Anadolu_17.jpg
Görsel kaydedildi: OutputImages\Anadolu_18.jpg
Görsel kaydedildi: OutputImages\Anadolu_19.jpg
Görsel kaydedildi: OutputIm

In [12]:
# === AnimeGAN Tanımı ===
class AnimeGAN:
    def __init__(self, model_path: str = '', downsize_ratio: float = 1.0):
        if not os.path.exists(model_path):
            raise Exception(f"Model bulunamadı: {model_path}")
        
        self.downsize_ratio = downsize_ratio
        providers = ['CUDAExecutionProvider'] if ort.get_device() == "GPU" else ['CPUExecutionProvider']
        self.ort_sess = ort.InferenceSession(model_path, providers=providers)

    def to_32s(self, x):
        return 256 if x < 256 else x - x % 32

    def process_frame(self, frame: np.ndarray, x32: bool = True) -> np.ndarray:
        h, w = frame.shape[:2]
        if x32:
            frame = cv2.resize(frame, (self.to_32s(int(w * self.downsize_ratio)), self.to_32s(int(h * self.downsize_ratio))))
        frame = frame.astype(np.float32) / 127.5 - 1.0
        return frame

    def post_process(self, frame: np.ndarray, wh: typing.Tuple[int, int]) -> np.ndarray:
        frame = (frame.squeeze() + 1.) / 2 * 255
        frame = frame.astype(np.uint8)
        frame = cv2.resize(frame, (wh[0], wh[1]))
        return frame

    def __call__(self, frame: np.ndarray) -> np.ndarray:
        image = self.process_frame(frame)
        outputs = self.ort_sess.run(None, {self.ort_sess._inputs_meta[0].name: np.expand_dims(image, axis=0)})
        return self.post_process(outputs[0], frame.shape[:2][::-1])


In [13]:
# Veri yolları
json_path = r"C:\Users\esmabetul\Desktop\YazılımGeliştirme\output_data.json"  # JSON dosyasının yolunu buraya yazın

class SignLanguageDataset(Dataset):  
    def __init__(self, json_path, image_size=(64, 64)):  
        with open(json_path, "r", encoding="utf-8") as f:  
            self.data = json.load(f)  
        
        self.image_paths = []  
        self.labels = []  
        self.label_map = {word: idx for idx, word in enumerate(self.data.keys())}  # Kelime - ID eşlemesi  

        for word, frames in self.data.items():  
            for frame in frames:  
                img_path = frame["image_path"].replace("\\", "/")  # Yol formatını düzeltme  
                print(f"Kontrol edilen dosya yolu: {img_path}")  # Hata ayıklama  
                if os.path.exists(img_path):  
                    img = cv2.imread(img_path)  
                    if img is not None:  
                        img = cv2.resize(img, image_size)  # Görselleri aynı boyuta getir  
                        img = img / 255.0  # Normalizasyon  
                        self.image_paths.append(img)  
                        self.labels.append(self.label_map[word])  
                        print(f"Resim kaydedildi: {img_path}")  # Yüklenen resmi belirtme  
                    else:  
                        print(f"Resim yüklenemedi (cv2.imread None): {img_path}")  
                else:  
                    print(f"Dosya mevcut değil: {img_path}")  

        self.image_paths = np.array(self.image_paths)  
        self.labels = np.array(self.labels)  

    def __len__(self):  
        return len(self.image_paths)  

    def __getitem__(self, idx):  
        img = self.image_paths[idx]  
        label = self.labels[idx]  
        img = np.transpose(img, (2, 0, 1))  # PyTorch tensor formatına çevirme (C, H, W)  
        img = torch.tensor(img, dtype=torch.float32)  
        label = torch.tensor(label, dtype=torch.long)  
        return img, label 




In [14]:
dataset = SignLanguageDataset(json_path)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Kontrol edilen dosya yolu: OutputImages/Anadolu_0.jpg
Resim kaydedildi: OutputImages/Anadolu_0.jpg
Kontrol edilen dosya yolu: OutputImages/Anadolu_1.jpg
Resim kaydedildi: OutputImages/Anadolu_1.jpg
Kontrol edilen dosya yolu: OutputImages/Anadolu_2.jpg
Resim kaydedildi: OutputImages/Anadolu_2.jpg
Kontrol edilen dosya yolu: OutputImages/Anadolu_3.jpg
Resim kaydedildi: OutputImages/Anadolu_3.jpg
Kontrol edilen dosya yolu: OutputImages/Anadolu_4.jpg
Resim kaydedildi: OutputImages/Anadolu_4.jpg
Kontrol edilen dosya yolu: OutputImages/Anadolu_5.jpg
Resim kaydedildi: OutputImages/Anadolu_5.jpg
Kontrol edilen dosya yolu: OutputImages/Anadolu_6.jpg
Resim kaydedildi: OutputImages/Anadolu_6.jpg
Kontrol edilen dosya yolu: OutputImages/Anadolu_7.jpg
Resim kaydedildi: OutputImages/Anadolu_7.jpg
Kontrol edilen dosya yolu: OutputImages/Anadolu_8.jpg
Resim kaydedildi: OutputImages/Anadolu_8.jpg
Kontrol edilen dosya yolu: OutputImages/Anadolu_9.jpg
Resim kaydedildi: OutputImages/Anadolu_9.jpg
Kontrol ed

In [15]:
# === CNN Modeli ===
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

# === Model Eğitimi ===
num_classes = len(dataset.label_map)
model = CNNModel(num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {acc:.2f}%")

# === Modeli ve Sınıf Sayısını Kaydet ===
torch.save({
    'model_state_dict': model.state_dict(),
    'num_classes': num_classes
}, 'sign_language_model.pth')


Epoch [1/10], Loss: 3.1118, Accuracy: 17.90%
Epoch [2/10], Loss: 1.5478, Accuracy: 51.15%
Epoch [3/10], Loss: 0.9223, Accuracy: 69.74%
Epoch [4/10], Loss: 0.6714, Accuracy: 76.88%
Epoch [5/10], Loss: 0.5438, Accuracy: 81.35%
Epoch [6/10], Loss: 0.4339, Accuracy: 84.06%
Epoch [7/10], Loss: 0.3587, Accuracy: 86.71%
Epoch [8/10], Loss: 0.3278, Accuracy: 87.67%
Epoch [9/10], Loss: 0.3096, Accuracy: 88.49%
Epoch [10/10], Loss: 0.2517, Accuracy: 90.76%


In [16]:
import cv2
import os
import json
import numpy as np

# AnimeGAN sınıfı tanımlı olmalı (önceden eklendiği gibi)
onnx_model_path = r'animegan.onnx'
animegan = AnimeGAN(model_path=onnx_model_path)  # AnimeGAN modeli yüklenir

# === JSON VERİSİ YÜKLE ===
with open("output_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)  # Bu satır eksikse hiçbir şey işlenmez!

# === İŞLEME FONKSİYONU ===
def preprocess_dataset(data, image_folder, output_folder="processed_images"):
    os.makedirs(output_folder, exist_ok=True)
    processed_data = {}

    for word, frames in data.items():
        processed_paths = []
        print(f"📦 Kelime işleniyor: {word}")
        for frame_info in frames:
            img_path = os.path.join(image_folder, os.path.basename(frame_info["image_path"]))
            print(f"📂 Dosya kontrol ediliyor: {img_path} - Var mı? {os.path.exists(img_path)}")
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                img = cv2.resize(img, (256, 256))
                anime_img = animegan(img)
                print("✨ AnimeGan filtresi uygulandı")
                save_path = os.path.join(output_folder, f"{word}_{len(processed_paths)}.jpg")
                cv2.imwrite(save_path, anime_img)
                processed_paths.append(save_path)

        processed_data[word] = processed_paths

    with open("processed_data.json", "w", encoding="utf-8") as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=4)

    print("✅ Görseller başarıyla işlendi ve 'processed_data.json' dosyasına kaydedildi.")

# === FONKSİYON ÇAĞRISI ===
preprocess_dataset(data, "OutputImages")


📦 Kelime işleniyor: Anadolu
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_0.jpg - Var mı? True
✨ AnimeGan filtresi uygulandı
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_1.jpg - Var mı? True
✨ AnimeGan filtresi uygulandı
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_2.jpg - Var mı? True
✨ AnimeGan filtresi uygulandı
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_3.jpg - Var mı? True
✨ AnimeGan filtresi uygulandı
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_4.jpg - Var mı? True
✨ AnimeGan filtresi uygulandı
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_5.jpg - Var mı? True
✨ AnimeGan filtresi uygulandı
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_6.jpg - Var mı? True
✨ AnimeGan filtresi uygulandı
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_7.jpg - Var mı? True
✨ AnimeGan filtresi uygulandı
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_8.jpg - Var mı? True
✨ AnimeGan filtresi uygulandı
📂 Dosya kontrol ediliyor: OutputImages\Anadolu_9.jpg - Var mı? True
✨ AnimeGan filtresi uy

In [19]:
import threading
import cv2
import time
import json
import speech_recognition as sr
import numpy as np

# === Türkçe Karakter Normalizasyon Fonksiyonu ===
def normalize_text(text):
    replacements = {
        "ç": "c", "ğ": "g", "ı": "i", "ö": "o", "ş": "s", "ü": "u",
        "Ç": "C", "Ğ": "G", "İ": "I", "Ö": "O", "Ş": "S", "Ü": "U"
    }
    for turkish_char, ascii_char in replacements.items():
        text = text.replace(turkish_char, ascii_char)
    return text

# JSON'dan işlenmiş veri yükleme
with open("processed_data.json", "r") as f:
    processed_data = json.load(f)

# Görüntüleri baştan belleğe al
cached_frames = {}
for word, paths in processed_data.items():
    cached_frames[word] = [cv2.imread(p) for p in paths]

def real_time_sign_language():
    words_queue = []
    stop_listening = threading.Event()

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Kamera açılamadı.")
        return

    # Pencere ayarları
    screen_width = 3000
    screen_height = 1200
    cv2.namedWindow("İşaret Dili Animasyonu", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("İşaret Dili Animasyonu", screen_width, screen_height)
    cv2.moveWindow("İşaret Dili Animasyonu", 0, 0)

    def listen_microphone():
        recognizer = sr.Recognizer()
        mic = sr.Microphone()

        with mic as source:
            recognizer.adjust_for_ambient_noise(source)

        while not stop_listening.is_set():
            with mic as source:
                try:
                    print(" Konuşabilirsiniz...")
                    audio = recognizer.listen(source, timeout=3, phrase_time_limit=5)
                    text = recognizer.recognize_google(audio, language="tr-TR")
                    words = text.split()
                    if words:
                        normalized_words = [normalize_text(w).capitalize() for w in words]
                        words_queue.extend(normalized_words)
                        print(f" Algılandı: {' '.join(normalized_words)}")
                except sr.UnknownValueError:
                    print(" Anlaşılamadı, lütfen tekrar edin...")
                except sr.RequestError:
                    print(" API hatası, internet bağlantınızı kontrol edin.")
                except:
                    continue

    threading.Thread(target=listen_microphone, daemon=True).start()

    print(" Kamera başlatıldı. Konuştuğunuz kelimelerin işaret dilini gösterecek. Çıkmak için 'q' tuşuna basın...")

    current_word = ""
    frame_list = []
    last_word = ""
    word_start_time = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Aynalama kaldırıldı (flip yok)
        user_frame = cv2.resize(frame, (screen_width, screen_height))
        anime_frame = np.zeros((500, 500, 3), dtype=np.uint8)

        # Yeni kelime kuyruğu varsa ve animasyon yoksa
        if words_queue and not frame_list:
            current_word = words_queue.pop(0)
            if current_word in cached_frames:
                frame_list = cached_frames[current_word]
                word_start_time = time.time()
                last_word = current_word  # Son kelimeyi güncelle
            else:
                print(f" Veri bulunamadı: {current_word}")
                frame_list = []

        # Animasyon varsa zamanla göster
        if frame_list:
            frame_count = len(frame_list)
            elapsed_time = (time.time() - word_start_time) * 75
            index = int(elapsed_time)

            if index < frame_count:
                anime_frame = cv2.resize(frame_list[index], (500, 500))
            else:
                frame_list = []
        elif last_word in cached_frames:
            # Eğer yeni kelime algılanmadıysa, son kelimenin ilk karesi gösterilsin
            anime_frame = cv2.resize(cached_frames[last_word][0], (500, 500))

        # Anime frame'i sağ alt köşeye yerleştir
        x_offset = screen_width - 500 - 20
        y_offset = screen_height - 500 - 20
        user_frame[y_offset:y_offset + 500, x_offset:x_offset + 500] = anime_frame

        cv2.imshow("İşaret Dili Animasyonu", user_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_listening.set()
            break

    cap.release()
    cv2.destroyAllWindows()
    print(" Sistem durduruldu.")


In [20]:
if __name__ == '__main__':
    json_path = r'processed_data.json'  # JSON veri dosyası yolu
    image_folder = r'processed_images'  # Görsellerin bulunduğu klasör
    onnx_model_path = r'animegan.onnx'  # AnimeGAN ONNX modeli yolu

    label_map = {word: idx for idx, word in enumerate(data.keys())}  # Etiket haritası oluşturulur

    num_classes = len(label_map)  
    model = CNNModel(num_classes) 
    # === Modeli Yükle ===
    checkpoint = torch.load("sign_language_model.pth", map_location=torch.device('cpu'))
    num_classes = checkpoint['num_classes']

    model = CNNModel(num_classes)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    animegan = AnimeGAN(model_path=onnx_model_path)  # AnimeGAN modeli yüklenir
    
   # Canlı sistem başlat
    real_time_sign_language()

    #sentence = record_and_recognize()  # Ses kaydı alınıp metne çevrilir

   # generate_video(sentence, model, label_map, data, animegan, image_folder)  # Video oluşturulur

 Kamera başlatıldı. Konuştuğunuz kelimelerin işaret dilini gösterecek. Çıkmak için 'q' tuşuna basın...
 Konuşabilirsiniz...
 Algılandı: Merhaba
 Konuşabilirsiniz...
 Algılandı: Istanbul'da Hava
 Veri bulunamadı: Istanbul'da
 Konuşabilirsiniz...
 Algılandı: Istanbul'da
 Veri bulunamadı: Istanbul'da
 Konuşabilirsiniz...
 Algılandı: Ne Vardi Hatirlamiyorum
 Veri bulunamadı: Ne
 Veri bulunamadı: Vardi
 Veri bulunamadı: Hatirlamiyorum
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Algılandı: Ayna Seyini Kaldirmis Mi Sen
 Veri bulunamadı: Ayna
 Veri bulunamadı: Seyini
 Veri bulunamadı: Kaldirmis
 Veri bulunamadı: Mi
 Veri bulunamadı: Sen
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Algılandı: Istanbul Hava Bugun
 Veri bulunamadı: Istanbul
 Konuşabilirsiniz...
 Algılandı: Istanbul'da Bir Sey Soracagim
 Veri bulunamadı: Istanbul'da
 Veri bulunamadı: Bir
 Veri bulunamadı: Sey
 Veri bulunamadı: Soracagim
 Konuşabilirsiniz...
 Algılandı: Kenar Kosesi Kesilmemis
 Veri bulunamadı: Ke